In [1]:
#!pip install sodapy
#!pip install schedule

In [2]:
import pandas as pd
from sodapy import Socrata
import schedule
import time

In [3]:
# Add data cleaning here too!
def data_acquisition():
    
    #Park Meter DataFrame
    client = Socrata("data.sfgov.org", app_token = 'T36UcTjsEvvBryJhihkNfsK6j')

    results = client.get("8vzz-qzz9", limit=100000000)

    park_meter_df = pd.DataFrame.from_records(results)

    park_meter_df.to_csv('park_meter.csv')


    #Meter Policies DataFrame
    client = Socrata("data.sfgov.org", app_token = 'T36UcTjsEvvBryJhihkNfsK6j')

    results = client.get("qq7v-hds4", limit=100000000)

    meter_policies_df = pd.DataFrame.from_records(results)

    meter_policies_df.to_csv('meter_policies.csv')
    
    

In [4]:
#data_acquisition()

In [5]:
# Run job on a specific day of the week
#schedule.every().tuesday.at("11:35").do(data_acquisition)

### Park Meter EDA/cleaning

In [6]:
park_meter_df = pd.read_csv('park_meter.csv', index_col = 0 )

In [7]:
park_meter_df.columns

Index(['objectid', 'parking_space_id', 'post_id', 'ms_pay_station_id',
       'ms_space_num', 'sensor_flag', 'on_offstreet_type', 'osp_id',
       'jurisdiction', 'pm_district_id', 'blockface_id', 'active_meter_flag',
       'reason_code', 'smart_meter_flag', 'meter_type', 'meter_vendor',
       'meter_model', 'cap_color', 'pco_beat', 'old_rate_area', 'street_id',
       'street_name', 'street_num', 'parity_digit_position',
       'street_seg_ctrln_id', 'orientation', 'longitude', 'latitude',
       'pmr_route', 'nfc_key', 'shape', ':@computed_region_jwn9_ihcz',
       ':@computed_region_6qbp_sg9q', ':@computed_region_qgnn_b9vv',
       ':@computed_region_26cr_cadq', ':@computed_region_ajp5_b2md',
       'collection_route', 'collection_subroute', 'collection_route_desc',
       'collection_subroute_desc', 'comments', 'legislation_ref', 'work_order',
       'legislation_dt'],
      dtype='object')

In [8]:
park_meter_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34240 entries, 0 to 34239
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   objectid                     34240 non-null  int64  
 1   parking_space_id             34240 non-null  float64
 2   post_id                      34240 non-null  object 
 3   ms_pay_station_id            34240 non-null  object 
 4   ms_space_num                 34240 non-null  float64
 5   sensor_flag                  34240 non-null  object 
 6   on_offstreet_type            34240 non-null  object 
 7   osp_id                       34240 non-null  float64
 8   jurisdiction                 34240 non-null  object 
 9   pm_district_id               34240 non-null  float64
 10  blockface_id                 34240 non-null  float64
 11  active_meter_flag            34240 non-null  object 
 12  reason_code                  34216 non-null  object 
 13  smart_meter_flag

In [9]:
park_meter_df[['osp_id', 'street_id', 'street_name', 'street_num']].head()

,osp_id,street_id,street_name,street_num
0,0.0,210.0,10TH ST,458.0
1,0.0,327.0,BROADWAY,805.0
2,0.0,847.0,JEFFERSON ST,422.0
3,0.0,686.0,TURK ST,544.0
4,0.0,568.0,MISSION ST,540.0


In [10]:
park_meter_df[['osp_id', 'street_id', 'street_name', 'street_num']][park_meter_df['street_name'] ==  'BROADWAY'].head(20)

,osp_id,street_id,street_name,street_num
1,0.0,327.0,BROADWAY,805.0
70,0.0,327.0,BROADWAY,1445.0
103,0.0,327.0,BROADWAY,724.0
218,0.0,327.0,BROADWAY,720.0
458,0.0,327.0,BROADWAY,1503.0
637,0.0,327.0,BROADWAY,404.0
918,0.0,327.0,BROADWAY,56.0
940,0.0,327.0,BROADWAY,338.0
964,0.0,327.0,BROADWAY,211.0
1258,0.0,327.0,BROADWAY,1443.0


In [11]:
park_meter_df.drop(['objectid', 'post_id', 'jurisdiction', 'pm_district_id', 
                    'blockface_id', 'reason_code', 'meter_vendor', 'meter_model',
                   'pco_beat', 'street_id', 'street_name', 'street_num', 'parity_digit_position',
                   'street_seg_ctrln_id', 'pmr_route', 'nfc_key', 'shape', ':@computed_region_jwn9_ihcz',
                   ':@computed_region_6qbp_sg9q', ':@computed_region_qgnn_b9vv', ':@computed_region_26cr_cadq',
                   ':@computed_region_ajp5_b2md', 'collection_route', 'collection_subroute', 
                    'collection_route_desc', 'comments', 'legislation_ref', 'work_order', 'legislation_dt',
                   'collection_subroute_desc', 'orientation', 'old_rate_area', 'osp_id']
                   , axis=1, inplace=True)

In [12]:
park_meter_df['smart_meter_flag'].describe()

count     34240
unique        2
top           Y
freq      31949
Name: smart_meter_flag, dtype: object

In [13]:
park_meter_df = park_meter_df[park_meter_df['smart_meter_flag'] == 'Y']

In [14]:
park_meter_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31949 entries, 0 to 34239
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   parking_space_id   31949 non-null  float64
 1   ms_pay_station_id  31949 non-null  object 
 2   ms_space_num       31949 non-null  float64
 3   sensor_flag        31949 non-null  object 
 4   on_offstreet_type  31949 non-null  object 
 5   active_meter_flag  31949 non-null  object 
 6   smart_meter_flag   31949 non-null  object 
 7   meter_type         31949 non-null  object 
 8   cap_color          31949 non-null  object 
 9   longitude          31949 non-null  float64
 10  latitude           31949 non-null  float64
dtypes: float64(4), object(7)
memory usage: 2.9+ MB


In [15]:
len(park_meter_df['parking_space_id'].unique())

31949

In [16]:
park_meter_df['ms_pay_station_id'].value_counts()

-            26832
892-00001       40
891-00002       32
891-00001       29
210-00004       22
             ...  
932-01000        1
936-01000        1
934-01000        1
935-02000        1
939-02000        1
Name: ms_pay_station_id, Length: 687, dtype: int64

In [17]:
park_meter_df['ms_space_num'].value_counts()

0.0     26914
5.0       253
1.0       248
3.0       242
4.0       234
        ...  
96.0        3
93.0        2
97.0        1
95.0        1
99.0        1
Name: ms_space_num, Length: 100, dtype: int64

In [18]:
park_meter_df = park_meter_df[['parking_space_id', 'longitude', 'latitude']]

In [19]:
park_meter_df.head()

,parking_space_id,longitude,latitude
0,100302.0,-122.409921,37.770380
1,100623.0,-122.410504,37.797220
3,101368.0,-122.418760,37.782263
4,102371.0,-122.398516,37.789129
6,103103.0,-122.449897,37.786787


In [20]:
park_meter_df.to_csv('park_meter_clean.csv', index=False)

### Meter Policies EDA/cleaning

In [98]:
meter_policies_df = pd.read_csv('meter_policies.csv', index_col = 0)

In [99]:
meter_policies_df

,revisiondate,startdate,enddate,postid,parkingspaceid,mspaystationid,spacenumber,dayofweek,starttime,endtime,scheduletype,pk_mapi,timelimitminutes,hourlyrate,capcolor
0,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,520-09150,101470,520-09001,15.0,Su,0:00,24:00,FREE,566431,NaN,NaN,NaN
1,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,571-09020,125289,-,0.0,Th,18:00,24:00,FREE,670622,NaN,NaN,NaN
2,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,571-09020,125289,-,0.0,Fr,0:00,4:30,FREE,670623,NaN,NaN,NaN
3,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,571-09020,125289,-,0.0,Fr,4:30,7:00,PRE,670624,120.0,NaN,NaN
4,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,571-09020,125289,-,0.0,Fr,7:00,12:00,OP,670625,120.0,1.0,Grey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029566,2015-02-05T10:19:35.000,2015-02-05T00:00:00.000,2200-12-31T23:59:59.000,891-02020,118136,891-00002,202.0,We,0:00,24:00,FREE,1008269,NaN,NaN,NaN
1029567,2015-02-05T10:19:35.000,2015-02-05T00:00:00.000,2200-12-31T23:59:59.000,891-02020,118136,891-00002,202.0,Th,0:00,24:00,FREE,1008270,NaN,NaN,NaN
1029568,2015-02-05T10:19:35.000,2015-02-05T00:00:00.000,2200-12-31T23:59:59.000,891-02020,118136,891-00002,202.0,Fr,0:00,24:00,FREE,1008271,NaN,NaN,NaN
1029569,2015-02-05T10:19:35.000,2015-02-05T00:00:00.000,2200-12-31T23:59:59.000,891-02020,118136,891-00002,202.0,Sa,0:00,24:00,FREE,1008272,NaN,NaN,NaN


In [100]:
meter_policies_df.columns

Index(['revisiondate', 'startdate', 'enddate', 'postid', 'parkingspaceid',
       'mspaystationid', 'spacenumber', 'dayofweek', 'starttime', 'endtime',
       'scheduletype', 'pk_mapi', 'timelimitminutes', 'hourlyrate',
       'capcolor'],
      dtype='object')

In [101]:
len(meter_policies_df['parkingspaceid'].unique())

31159

In [102]:
meter_policies_df['scheduletype'].unique()

array(['FREE', 'PRE', 'OP', 'TOW', 'ALT'], dtype=object)

In [103]:
meter_policies_df['timelimitminutes'].unique()

array([  nan,  120.,  240.,   15.,   30., 1440.,   60.,   90.,  720.,
        180.,  540.,  600.,    0.])

In [104]:
meter_policies_df[meter_policies_df['parkingspaceid'] == 101470]

,revisiondate,startdate,enddate,postid,parkingspaceid,mspaystationid,spacenumber,dayofweek,starttime,endtime,scheduletype,pk_mapi,timelimitminutes,hourlyrate,capcolor
0,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,520-09150,101470,520-09001,15.0,Su,0:00,24:00,FREE,566431,NaN,NaN,NaN
2663,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,520-09150,101470,520-09001,15.0,Mo,0:00,4:30,FREE,566405,NaN,NaN,NaN
2664,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,520-09150,101470,520-09001,15.0,Mo,4:30,7:00,PRE,566406,30.0,NaN,NaN
2665,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,520-09150,101470,520-09001,15.0,Mo,7:00,18:00,OP,566407,30.0,3.25,Yellow
2666,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,520-09150,101470,520-09001,15.0,Mo,18:00,24:00,FREE,566408,NaN,NaN,NaN
2667,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,520-09150,101470,520-09001,15.0,Tu,0:00,4:30,FREE,566409,NaN,NaN,NaN
2668,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,520-09150,101470,520-09001,15.0,Tu,4:30,7:00,PRE,566410,30.0,NaN,NaN
2669,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,520-09150,101470,520-09001,15.0,Tu,7:00,18:00,OP,566411,30.0,3.25,Yellow
2670,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,520-09150,101470,520-09001,15.0,Tu,18:00,24:00,FREE,566412,NaN,NaN,NaN
2671,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,520-09150,101470,520-09001,15.0,We,0:00,4:30,FREE,566413,NaN,NaN,NaN


In [105]:
len(set(meter_policies_df['parkingspaceid']) & set(park_meter_df['parking_space_id']))

31147

In [106]:
meter_policies_df[meter_policies_df['enddate'] < '2022']

,revisiondate,startdate,enddate,postid,parkingspaceid,mspaystationid,spacenumber,dayofweek,starttime,endtime,scheduletype,pk_mapi,timelimitminutes,hourlyrate,capcolor


In [107]:
meter_policies_df[meter_policies_df.duplicated(subset=['startdate', 'parkingspaceid', 'dayofweek', 'starttime'])]

,revisiondate,startdate,enddate,postid,parkingspaceid,mspaystationid,spacenumber,dayofweek,starttime,endtime,scheduletype,pk_mapi,timelimitminutes,hourlyrate,capcolor
915245,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,Mo,0:00,4:30,FREE,95420,NaN,NaN,NaN
915253,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,Mo,4:30,7:00,PRE,95421,1440.0,NaN,NaN
915256,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,Mo,7:00,18:00,OP,95422,1440.0,0.65,Black
915258,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,Mo,18:00,24:00,FREE,95423,NaN,NaN,NaN
915260,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,Tu,0:00,4:30,FREE,95424,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022835,2016-01-05T11:53:53.000,2014-08-08T00:00:00.000,2200-12-31T23:59:59.000,891-02120,105109,891-00002,212.0,Mo,0:00,24:00,OP,1008337,1440.0,0.21,Purple
1022836,2016-01-05T11:53:53.000,2014-08-08T00:00:00.000,2200-12-31T23:59:59.000,891-02120,105109,891-00002,212.0,Tu,0:00,24:00,OP,1008338,1440.0,0.21,Purple
1022837,2016-01-05T11:53:53.000,2014-08-08T00:00:00.000,2200-12-31T23:59:59.000,891-02120,105109,891-00002,212.0,We,0:00,24:00,OP,1008339,1440.0,0.21,Purple
1022838,2016-01-05T11:53:53.000,2014-08-08T00:00:00.000,2200-12-31T23:59:59.000,891-02120,105109,891-00002,212.0,Th,0:00,24:00,OP,1008340,1440.0,0.21,Purple


In [108]:
meter_policies_df[meter_policies_df['parkingspaceid'] == 127279].sort_values(['revisiondate'])

,revisiondate,startdate,enddate,postid,parkingspaceid,mspaystationid,spacenumber,dayofweek,starttime,endtime,scheduletype,pk_mapi,timelimitminutes,hourlyrate,capcolor
916892,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,Su,0:00,24:00,FREE,95444,NaN,NaN,NaN
915253,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,Mo,4:30,7:00,PRE,95421,1440.0,NaN,NaN
915256,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,Mo,7:00,18:00,OP,95422,1440.0,0.65,Black
915258,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,Mo,18:00,24:00,FREE,95423,NaN,NaN,NaN
915260,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,Tu,0:00,4:30,FREE,95424,NaN,NaN,NaN
915263,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,Tu,4:30,7:00,PRE,95425,1440.0,NaN,NaN
915265,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,Tu,7:00,18:00,OP,95426,1440.0,0.65,Black
915274,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,Tu,18:00,24:00,FREE,95427,NaN,NaN,NaN
915280,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,We,0:00,4:30,FREE,95428,NaN,NaN,NaN
915284,2020-06-23T10:08:40.000,2020-07-06T00:00:00.000,2200-12-31T23:59:59.000,211-00510,127279,211-00005,51.0,We,4:30,7:00,PRE,95429,1440.0,NaN,NaN


In [109]:
meter_policies_df.drop(['postid', 'mspaystationid','spacenumber', 'pk_mapi', 'capcolor'], 
                       axis=1, inplace=True)

In [110]:
meter_policies_df.head()

,revisiondate,startdate,enddate,parkingspaceid,dayofweek,starttime,endtime,scheduletype,timelimitminutes,hourlyrate
0,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,101470,Su,0:00,24:00,FREE,NaN,NaN
1,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,125289,Th,18:00,24:00,FREE,NaN,NaN
2,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,125289,Fr,0:00,4:30,FREE,NaN,NaN
3,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,125289,Fr,4:30,7:00,PRE,120.0,NaN
4,2021-12-13T12:54:14.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,125289,Fr,7:00,12:00,OP,120.0,1.0


In [111]:
meter_policies_df.sort_values(['parkingspaceid', 'revisiondate', 'startdate', 'dayofweek', 'starttime'], 
                              inplace=True)

In [112]:
meter_policies_df.drop_duplicates(subset=['parkingspaceid', 'dayofweek', 'starttime'],
                                 keep = 'last', inplace=True)

In [113]:
meter_policies_df.head()

,revisiondate,startdate,enddate,parkingspaceid,dayofweek,starttime,endtime,scheduletype,timelimitminutes,hourlyrate
421578,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Fr,0:00,4:30,FREE,NaN,NaN
421582,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Fr,18:00,24:00,FREE,NaN,NaN
421579,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Fr,4:30,9:00,PRE,120.0,NaN
421580,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Fr,9:00,18:00,OP,120.0,2.25
421561,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Mo,0:00,4:30,FREE,NaN,NaN


In [114]:
meter_policies_df[meter_policies_df.duplicated(subset=['parkingspaceid', 'dayofweek', 'starttime'])]

,revisiondate,startdate,enddate,parkingspaceid,dayofweek,starttime,endtime,scheduletype,timelimitminutes,hourlyrate


In [115]:
meter_policies_df[meter_policies_df['parkingspaceid'] == 100108].head(50)

,revisiondate,startdate,enddate,parkingspaceid,dayofweek,starttime,endtime,scheduletype,timelimitminutes,hourlyrate
871831,2020-12-21T14:33:27.000,2021-01-04T00:00:00.000,2200-12-31T23:59:59.000,100108,Fr,0:00,4:30,FREE,NaN,NaN
871835,2020-12-21T14:33:27.000,2021-01-04T00:00:00.000,2200-12-31T23:59:59.000,100108,Fr,12:00,15:00,OP,120.0,2.75
871836,2020-12-21T14:33:27.000,2021-01-04T00:00:00.000,2200-12-31T23:59:59.000,100108,Fr,15:00,18:00,OP,120.0,1.25
871837,2020-12-21T14:33:27.000,2021-01-04T00:00:00.000,2200-12-31T23:59:59.000,100108,Fr,18:00,24:00,OP,120.0,1.50
871833,2020-12-21T14:33:27.000,2021-01-04T00:00:00.000,2200-12-31T23:59:59.000,100108,Fr,4:30,9:00,PRE,120.0,NaN
871834,2020-12-21T14:33:27.000,2021-01-04T00:00:00.000,2200-12-31T23:59:59.000,100108,Fr,9:00,12:00,OP,120.0,1.25
871806,2020-12-21T14:33:27.000,2021-01-04T00:00:00.000,2200-12-31T23:59:59.000,100108,Mo,0:00,4:30,FREE,NaN,NaN
871809,2020-12-21T14:33:27.000,2021-01-04T00:00:00.000,2200-12-31T23:59:59.000,100108,Mo,12:00,15:00,OP,120.0,2.75
871810,2020-12-21T14:33:27.000,2021-01-04T00:00:00.000,2200-12-31T23:59:59.000,100108,Mo,15:00,18:00,OP,120.0,1.25
871811,2020-12-21T14:33:27.000,2021-01-04T00:00:00.000,2200-12-31T23:59:59.000,100108,Mo,18:00,24:00,OP,120.0,1.50


In [116]:
meter_policies_df[meter_policies_df['startdate'] > '2021-12-14']

,revisiondate,startdate,enddate,parkingspaceid,dayofweek,starttime,endtime,scheduletype,timelimitminutes,hourlyrate


In [117]:
meter_policies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1011185 entries, 421578 to 3515
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   revisiondate      1011185 non-null  object 
 1   startdate         1011185 non-null  object 
 2   enddate           1011185 non-null  object 
 3   parkingspaceid    1011185 non-null  int64  
 4   dayofweek         1011185 non-null  object 
 5   starttime         1011185 non-null  object 
 6   endtime           1011185 non-null  object 
 7   scheduletype      1011185 non-null  object 
 8   timelimitminutes  585758 non-null   float64
 9   hourlyrate        406245 non-null   float64
dtypes: float64(2), int64(1), object(7)
memory usage: 84.9+ MB


In [118]:
meter_policies_df['endtime'].replace({'24:00': '23:59'}, inplace=True)
meter_policies_df['starttime'].replace({'24:00': '23:59'}, inplace=True)

In [119]:
meter_policies_df['starttime'] = pd.to_datetime(meter_policies_df['starttime']).dt.time
meter_policies_df['endtime'] = pd.to_datetime(meter_policies_df['endtime']).dt.time

In [121]:
meter_policies_df.head()

,revisiondate,startdate,enddate,parkingspaceid,dayofweek,starttime,endtime,scheduletype,timelimitminutes,hourlyrate
421578,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Fr,00:00:00,04:30:00,FREE,NaN,NaN
421582,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Fr,18:00:00,23:59:00,FREE,NaN,NaN
421579,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Fr,04:30:00,09:00:00,PRE,120.0,NaN
421580,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Fr,09:00:00,18:00:00,OP,120.0,2.25
421561,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Mo,00:00:00,04:30:00,FREE,NaN,NaN


In [123]:
meter_policies_df['timelimitminutes'] = meter_policies_df['timelimitminutes'].fillna(0)
meter_policies_df['hourlyrate'] = meter_policies_df['hourlyrate'].fillna(0)

In [124]:
meter_policies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1011185 entries, 421578 to 3515
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   revisiondate      1011185 non-null  object 
 1   startdate         1011185 non-null  object 
 2   enddate           1011185 non-null  object 
 3   parkingspaceid    1011185 non-null  int64  
 4   dayofweek         1011185 non-null  object 
 5   starttime         1011185 non-null  object 
 6   endtime           1011185 non-null  object 
 7   scheduletype      1011185 non-null  object 
 8   timelimitminutes  1011185 non-null  float64
 9   hourlyrate        1011185 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 84.9+ MB


In [125]:
meter_policies_df.head()

,revisiondate,startdate,enddate,parkingspaceid,dayofweek,starttime,endtime,scheduletype,timelimitminutes,hourlyrate
421578,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Fr,00:00:00,04:30:00,FREE,0.0,0.00
421582,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Fr,18:00:00,23:59:00,FREE,0.0,0.00
421579,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Fr,04:30:00,09:00:00,PRE,120.0,0.00
421580,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Fr,09:00:00,18:00:00,OP,120.0,2.25
421561,2021-11-29T16:19:10.000,2021-12-06T00:00:00.000,2200-12-31T23:59:59.000,100001,Mo,00:00:00,04:30:00,FREE,0.0,0.00


In [94]:
meter_policies_df.to_csv('meter_policies_clean.csv', index=False)